In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv("housing.csv", low_memory=False)


In [4]:
pd.set_option('display.max_rows', 120)
print(df.dtypes)

Property Index Number                         object
Assessment Triad                              object
Property Class                                 int64
Property Address                              object
Property Apartment Num.                       object
Property City                                 object
Township Name                                 object
Township Code                                  int64
Neighborhood Code                              int64
Township Neighborhood Only                     int64
Modeling Group                                object
Proportion Assessed                          float64
Key PIN                                       object
Multi Code Indicator                           int64
Multi Code                                   float64
Number of Active 288s (HIEs)                 float64
Number of Ended 288s (HIEs)                  float64
Sale Date                                     object
Sale Price                                   f

In [5]:
df2 = df[
    [
        "Property Index Number",
        "Property Class",
        "Township Name",
        "Number of Active 288s (HIEs)",
        "Number of Ended 288s (HIEs)",
        "Sale Price",
        "Land Square Feet",
        "Large Home Indicator",
        "Age",
        "Apartments",
        "Wall Material",
        "Roof Material",
        "Rooms",
        "Bedrooms",
        "Basement",
        "Basement Finish",
        "Central Heating",
        "Central Air Conditioning",
        "Fireplaces",
        "Attic Type",
        "Full Baths",
        "Half Baths",
        "Cathedral Ceiling",
        "Garage 1 Size",
        "Garage 1 Material",
        "Garage 1 Attached",
        "Garage 1 Area",
        "Building Square Feet",
        "Repair Condition",
        "Attic Finish",
        "Renovation",
        "Porch",
        # "Property Zip Code",
        "Tract GEOID",
        "O'Hare Noise Indicator",
        "FEMA Floodplain",
        "Flood Risk Factor",
        "Flood Risk Direction",
        "Road Proximity < 100 Feet",
        "Road Proximity 101 - 300 Feet",
        "Elementary/Middle School District",
        "High School District",
        "Longitude",
        "Latitude",
        # "geoposition",
        "Tax Rate",
        "Tract Median Income",
        "Modeled Sale Year",
        "Modeled Sale Month",
        "Modeled Sale Day",
        "Modeled Sale During School Year",
        "Modeled Sale During Holidays",
    ]
]

df2.to_csv("housing_cleaned.csv")

In [6]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import Ridge

# Import Dataset

link to dataset: https://datacatalog.cookcountyil.gov/Property-Taxation/Assessor-Archived-05-11-2022-Residential-Modeling-/8f9d-wy2d


In [7]:
print(df2.shape)
df2.dropna(subset=["Sale Price"], inplace=True)
print(df2.shape)
# df2.to_csv("housing2.csv", index=False)

(678656, 50)
(78156, 50)


/var/folders/w8/480pcx_d3bx9y45bcf4fz7800000gn/T/ipykernel_60329/1126815284.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.dropna(subset=["Sale Price"], inplace=True)


In [8]:
if 'Property Index Number' in df2.columns:
	df2.set_index('Property Index Number', inplace=True)
	
for x in df2.columns:
	# print(x)
	if df2[x].dtype == 'object':
		df2[x].replace(df2[x].unique(), range(len(df2[x].unique())), inplace=True)
		df2[x] = df2[x].astype('category')
	

	
df2 = df2.dropna()




/var/folders/w8/480pcx_d3bx9y45bcf4fz7800000gn/T/ipykernel_60329/3112742252.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[x].replace(df2[x].unique(), range(len(df2[x].unique())), inplace=True)
/var/folders/w8/480pcx_d3bx9y45bcf4fz7800000gn/T/ipykernel_60329/3112742252.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[x] = df2[x].astype('category')


In [9]:
lm = Ridge(fit_intercept=True)
sfs = SequentialFeatureSelector(lm, n_features_to_select='auto', direction='forward', tol=0.001)
sfs.fit(df2.drop('Sale Price', axis=1), df2['Sale Price'])

SequentialFeatureSelector(estimator=Ridge(), n_features_to_select='auto',
                          tol=0.001)

In [10]:
sfs.get_support()
sfs.get_feature_names_out()

array(['Property Class', 'Number of Active 288s (HIEs)',
       'Number of Ended 288s (HIEs)', 'Large Home Indicator',
       'Wall Material', 'Roof Material', 'Fireplaces', 'Full Baths',
       'Cathedral Ceiling', 'Repair Condition', 'Renovation',
       "O'Hare Noise Indicator", 'Flood Risk Factor',
       'Elementary/Middle School District', 'Tract Median Income'],
      dtype=object)

In [11]:
X = sfs.transform(df2.drop('Sale Price', axis=1))
Y = df2['Sale Price']